### INPUT Number of rows for aggregated dataset

In [82]:
grouped = 100

In [83]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import os

# List all files under the input directory
#path = '/kaggle'
path = '/tmp/working'
input_path = path + '/input/gearbox-fault-diagnosis/'


# Read datasets
## Healthy gearbox

In [84]:
# Healthy gearbox
# ---------------
h30hz0  = pd.read_csv(input_path + "/Healthy/h30hz0.csv")
h30hz10 = pd.read_csv(input_path + "/Healthy/h30hz10.csv")
h30hz20 = pd.read_csv(input_path + "/Healthy/h30hz20.csv")
h30hz30 = pd.read_csv(input_path + "/Healthy/h30hz30.csv")
h30hz40 = pd.read_csv(input_path + "/Healthy/h30hz40.csv")
h30hz50 = pd.read_csv(input_path + "/Healthy/h30hz50.csv")
h30hz60 = pd.read_csv(input_path + "/Healthy/h30hz60.csv")
h30hz70 = pd.read_csv(input_path + "/Healthy/h30hz70.csv")
h30hz80 = pd.read_csv(input_path + "/Healthy/h30hz80.csv")
h30hz90 = pd.read_csv(input_path + "/Healthy/h30hz90.csv")


# Broken gearbox

In [85]:
# Broken gearbox
# --------------
b30hz0  = pd.read_csv(input_path + "/BrokenTooth/b30hz0.csv")
b30hz10 = pd.read_csv(input_path + "/BrokenTooth/b30hz10.csv")
b30hz20 = pd.read_csv(input_path + "/BrokenTooth/b30hz20.csv")
b30hz30 = pd.read_csv(input_path + "/BrokenTooth/b30hz30.csv")
b30hz40 = pd.read_csv(input_path + "/BrokenTooth/b30hz40.csv")
b30hz50 = pd.read_csv(input_path + "/BrokenTooth/b30hz50.csv")
b30hz60 = pd.read_csv(input_path + "/BrokenTooth/b30hz60.csv")
b30hz70 = pd.read_csv(input_path + "/BrokenTooth/b30hz70.csv")
b30hz80 = pd.read_csv(input_path + "/BrokenTooth/b30hz80.csv")
b30hz90 = pd.read_csv(input_path + "/BrokenTooth/b30hz90.csv")

# Compute aggregated datasets
## Function to compute aggregated features: standard deviation

In [86]:
def stdev_features(df, grouped, load, outcome):
    #### Create empty dataframe with columns a1,a2,a3,a4
    df_result = pd.DataFrame( [ np.zeros(len(df.columns)-2) ],columns= df.columns[:-2])

    #### Aggregate in groups of nrows computing the standard deviation
    # Remove load & failure columns, keeping only a1,a2,a3,a4
    stdev_lenght=len(df.columns)-2
    
    #### Compute number of rows of the aggregated dataframe
    nrows_raw = len(df.index)
    nrows = math.floor(nrows_raw / grouped)
    nrows_dropped = nrows_raw - nrows*grouped
    print("nrows_raw=", nrows_raw, "   nrows=", nrows, "   Number of dropped rows of grouped= ", nrows_dropped/grouped*100,"%\n")
    
    # Iterate every 'grouped' rows and compute stdev per column
    for i in range(nrows):
      df_group = df.iloc[i*grouped:i*grouped+grouped,:]
      df_stdev = pd.DataFrame(df_group.std()).transpose()
      # Remove load & failure columns
      df_stdev=df_stdev.iloc[:,:stdev_lenght]
      # Add row of calculated stdev
      df_result = df_result.append(df_stdev, ignore_index=True)

      #print ("i*grouped TO i*grouped+grouped", i*grouped, i*grouped+grouped)
      #print ("row, df_stdev=\n", row, df_stdev.iloc[:,:])
      #print ("df_result=\n", df_result)
      #print("row", i, "\n", df_group)


    # Remove the first row (it was the seed of zeros for initializing df_result)
    #print(df_result)
    df_result = df_result.iloc[1:,:]
    # Add the column for 'load'
    df_result['load'] = load*np.ones((nrows,1))

    # Add the column for 'failure'
    failure = np.ones((nrows,1))
    df_result['failure'] = outcome
    print(df_result)
    
    return df_result
    

## Load= 0%

In [87]:
# [H.0] DATASET BROKEN, 0% LOAD
load = 0

# CREATE THE COLUMN FOR 'load'
## Expressed as %
b30hz0['load'] = load*np.ones((len(b30hz0.index),1))

# CREATE THE COLUMN FOR 'failure'
##   0 is healthy
##   1 is broken
failure = 1
failureArray = np.ones((len(b30hz0.index),1))
b30hz0['failure'] = failureArray
#b30hz0.describe()

b30hz0_stdev= stdev_features (df = b30hz0, grouped= grouped, load= load, outcome= failure)
#b30hz0_stdev.describe()

nrows_raw= 88320    nrows= 883    Number of dropped rows of grouped=  20.0 %

           a1        a2        a3        a4  load  failure
1    2.773743  3.625441  3.053217  2.479670   0.0        1
2    3.009118  3.566201  2.854556  2.597574   0.0        1
3    2.258893  3.189251  1.971267  1.726624   0.0        1
4    2.486789  2.758437  2.658690  2.068924   0.0        1
5    3.245092  3.051541  2.536531  2.085758   0.0        1
..        ...       ...       ...       ...   ...      ...
879  4.655225  3.947581  3.061973  4.061803   0.0        1
880  3.364573  5.013792  3.509107  3.431726   0.0        1
881  3.749922  3.874292  3.278999  3.746721   0.0        1
882  2.389255  2.278619  1.591716  2.295268   0.0        1
883  4.544790  4.695638  4.655669  3.433364   0.0        1

[883 rows x 6 columns]


In [88]:
# [H.0] DATASET HEALTHY, 0% LOAD
load = 0

# CREATE THE COLUMN FOR 'load'
## Expressed as %
h30hz0['load'] = load*np.ones((len(h30hz0.index),1))

# CREATE THE COLUMN FOR 'failure'
failure = 0
failureArray = np.ones((len(h30hz0.index),1))
h30hz0['failure'] = failureArray

h30hz0_stdev= stdev_features (df = h30hz0, grouped= grouped, load= load, outcome= failure)
#b30hz0_stdev.describe()

nrows_raw= 88832    nrows= 888    Number of dropped rows of grouped=  32.0 %

           a1        a2        a3        a4  load  failure
1    4.518469  2.627219  3.129601  3.046165   0.0        0
2    3.463637  2.616907  2.286667  2.389276   0.0        0
3    4.461867  2.833214  2.413283  2.977934   0.0        0
4    4.381037  4.703040  2.668196  2.650610   0.0        0
5    3.880276  3.054690  2.502711  2.147417   0.0        0
..        ...       ...       ...       ...   ...      ...
884  2.119644  1.351392  1.524543  1.442983   0.0        0
885  3.644520  2.551445  2.531753  2.083169   0.0        0
886  3.096537  2.772343  1.819461  1.997667   0.0        0
887  4.810296  2.917624  2.365434  2.678131   0.0        0
888  3.814070  2.744130  2.529900  2.127945   0.0        0

[888 rows x 6 columns]


## Load= 10%

In [89]:
# [H.1] DATASET BROKEN, 10% LOAD
load = 10

# CREATE THE COLUMN FOR 'load'
## Expressed as %
b30hz10['load'] = load*np.ones((len(b30hz10.index),1))

# CREATE THE COLUMN FOR 'failure'
##   0 is healthy
##   1 is broken
failure = 1
failureArray = np.ones((len(b30hz10.index),1))
b30hz10['failure'] = failureArray
#b30hz10.describe()

b30hz10_stdev= stdev_features (df = b30hz10, grouped= grouped, load= load, outcome= failure)
#b30hz10_stdev.describe()

nrows_raw= 111616    nrows= 1116    Number of dropped rows of grouped=  16.0 %

            a1        a2        a3        a4  load  failure
1     3.947241  2.625565  3.075201  2.759145  10.0        1
2     3.768801  3.270266  2.846681  3.223817  10.0        1
3     3.318944  4.121886  2.990541  3.178636  10.0        1
4     5.918385  4.390473  3.856991  4.703490  10.0        1
5     2.621164  3.098193  2.674376  2.385791  10.0        1
...        ...       ...       ...       ...   ...      ...
1112  2.199512  3.386571  2.376948  2.548087  10.0        1
1113  2.590362  3.094107  2.297680  2.346342  10.0        1
1114  3.397182  3.866719  3.163785  3.127293  10.0        1
1115  2.893202  4.044202  2.679493  2.321979  10.0        1
1116  1.981846  2.945338  2.128224  1.803724  10.0        1

[1116 rows x 6 columns]


In [90]:
# [H.1] DATASET HEALTHY, 10% LOAD
load = 10

# CREATE THE COLUMN FOR 'load'
## Expressed as %
h30hz10['load'] = load*np.ones((len(h30hz10.index),1))

# CREATE THE COLUMN FOR 'failure'
failure = 0
failureArray = np.ones((len(h30hz10.index),1))
h30hz10['failure'] = failureArray

h30hz10_stdev= stdev_features (df = h30hz10, grouped= grouped, load= load, outcome= failure)
#b30hz10_stdev.describe()

nrows_raw= 92928    nrows= 929    Number of dropped rows of grouped=  28.000000000000004 %

            a1        a2        a3        a4  load  failure
1    10.138598  4.113320  4.783741  4.474848  10.0        0
2     7.550895  6.508534  3.872753  4.031962  10.0        0
3     5.142263  5.157923  3.725511  3.112694  10.0        0
4     4.592986  2.735169  3.135584  2.825671  10.0        0
5     4.486037  3.127276  2.870951  2.695032  10.0        0
..         ...       ...       ...       ...   ...      ...
925   5.885659  3.794135  3.414226  3.333931  10.0        0
926   6.896978  4.911135  3.771213  3.553566  10.0        0
927   6.372834  6.325193  3.777180  3.369915  10.0        0
928   5.415537  5.092150  3.459654  3.222893  10.0        0
929   4.474746  2.769132  2.779264  3.096695  10.0        0

[929 rows x 6 columns]


## Load= 20%

In [91]:
# [H.2] DATASET BROKEN, 20% LOAD
load = 20

# CREATE THE COLUMN FOR 'load'
## Expressed as %
b30hz20['load'] = load*np.ones((len(b30hz20.index),1))

# CREATE THE COLUMN FOR 'failure'
##   0 is healthy
##   1 is broken
failure = 1
failureArray = np.ones((len(b30hz20.index),1))
b30hz20['failure'] = failureArray
#b30hz20.describe()

b30hz20_stdev= stdev_features (df = b30hz20, grouped= grouped, load= load, outcome= failure)
#b30hz20_stdev.describe()

nrows_raw= 114432    nrows= 1144    Number of dropped rows of grouped=  32.0 %

            a1        a2        a3        a4  load  failure
1     2.924137  5.003475  3.886632  4.737741  20.0        1
2     2.648921  2.820215  2.994127  2.860806  20.0        1
3     3.610350  2.969249  2.743845  4.603176  20.0        1
4     6.321876  4.448391  4.400342  4.199627  20.0        1
5     3.249063  3.808483  3.581892  4.354598  20.0        1
...        ...       ...       ...       ...   ...      ...
1140  2.845084  3.029629  2.511096  2.488121  20.0        1
1141  2.373844  4.022596  3.044352  3.061497  20.0        1
1142  3.061555  3.817103  2.591231  2.746654  20.0        1
1143  3.408097  4.194282  3.413312  4.026020  20.0        1
1144  3.823588  3.822165  3.614803  3.805729  20.0        1

[1144 rows x 6 columns]


In [92]:
# [H.2] DATASET HEALTHY, 20% LOAD
load = 20

# CREATE THE COLUMN FOR 'load'
## Expressed as %
h30hz20['load'] = load*np.ones((len(h30hz20.index),1))

# CREATE THE COLUMN FOR 'failure'
failure = 0
failureArray = np.ones((len(h30hz20.index),1))
h30hz20['failure'] = failureArray

h30hz20_stdev= stdev_features (df = h30hz20, grouped= grouped, load= load, outcome= failure)
#b30hz20_stdev.describe()

nrows_raw= 108544    nrows= 1085    Number of dropped rows of grouped=  44.0 %

             a1        a2        a3        a4  load  failure
1      5.487624  3.360786  3.229484  3.120885  20.0        0
2     11.425865  4.412819  5.381959  4.509242  20.0        0
3      7.846365  6.306435  4.572026  4.374290  20.0        0
4      5.836613  3.534423  3.558668  3.548425  20.0        0
5      4.122627  2.249694  2.250503  2.882324  20.0        0
...         ...       ...       ...       ...   ...      ...
1081   2.546591  1.590065  1.990194  1.773325  20.0        0
1082   4.302383  3.084486  2.749144  2.380652  20.0        0
1083   3.871073  3.449995  2.670193  2.639361  20.0        0
1084   4.782154  2.943181  2.116463  2.130002  20.0        0
1085   3.941571  2.712904  2.984194  2.248856  20.0        0

[1085 rows x 6 columns]


## Load= 30%

In [93]:
# [H.3] DATASET BROKEN, 30% LOAD
load = 30

# CREATE THE COLUMN FOR 'load'
## Expressed as %
b30hz30['load'] = load*np.ones((len(b30hz30.index),1))

# CREATE THE COLUMN FOR 'failure'
##   0 is healthy
##   1 is broken
failure = 1
failureArray = np.ones((len(b30hz30.index),1))
b30hz30['failure'] = failureArray
#b30hz30.describe()

b30hz30_stdev= stdev_features (df = b30hz30, grouped= grouped, load= load, outcome= failure)
#b30hz30_stdev.describe()

nrows_raw= 89856    nrows= 898    Number of dropped rows of grouped=  56.00000000000001 %

           a1        a2        a3        a4  load  failure
1    5.178408  5.534030  5.368494  5.665968  30.0        1
2    4.277515  4.039146  4.119412  4.148621  30.0        1
3    2.874628  3.474401  3.175718  2.920996  30.0        1
4    3.390684  3.455118  2.970112  3.334487  30.0        1
5    3.868169  3.979720  3.621905  4.327168  30.0        1
..        ...       ...       ...       ...   ...      ...
894  5.975786  3.860299  4.174251  4.349155  30.0        1
895  4.934639  4.614853  4.536547  5.837843  30.0        1
896  4.078120  4.067854  3.883442  3.985121  30.0        1
897  4.037747  3.578898  3.573310  4.393785  30.0        1
898  4.181590  3.476764  3.603086  2.896070  30.0        1

[898 rows x 6 columns]


In [94]:
# [H.3] DATASET HEALTHY, 30% LOAD
load = 30

# CREATE THE COLUMN FOR 'load'
## Expressed as %
h30hz30['load'] = load*np.ones((len(h30hz30.index),1))

# CREATE THE COLUMN FOR 'failure'
failure = 0
failureArray = np.ones((len(h30hz30.index),1))
h30hz30['failure'] = failureArray

h30hz30_stdev= stdev_features (df = h30hz30, grouped= grouped, load= load, outcome= failure)
#b30hz30_stdev.describe()

nrows_raw= 106240    nrows= 1062    Number of dropped rows of grouped=  40.0 %

             a1        a2        a3        a4  load  failure
1      6.722006  3.798253  3.494062  4.340654  30.0        0
2      6.969772  2.950190  3.933455  4.042469  30.0        0
3     11.202765  4.641950  5.703556  4.750020  30.0        0
4      6.208754  5.846026  3.479599  4.163089  30.0        0
5      7.609760  5.184631  4.541050  4.068528  30.0        0
...         ...       ...       ...       ...   ...      ...
1058  10.205741  4.588561  5.385186  4.985253  30.0        0
1059   8.113069  8.247422  5.233717  4.560866  30.0        0
1060   5.467817  5.006407  4.275523  3.899923  30.0        0
1061   5.659547  3.137832  3.594448  3.691319  30.0        0
1062   7.130998  3.690707  4.590844  4.268382  30.0        0

[1062 rows x 6 columns]


## Load= 40%

In [95]:
# [H.4] DATASET BROKEN, 40% LOAD
load = 40

# CREATE THE COLUMN FOR 'load'
## Expressed as %
b30hz40['load'] = load*np.ones((len(b30hz40.index),1))

# CREATE THE COLUMN FOR 'failure'
##   0 is healthy
##   1 is broken
failure = 1
failureArray = np.ones((len(b30hz40.index),1))
b30hz40['failure'] = failureArray
#b30hz40.describe()

b30hz40_stdev= stdev_features (df = b30hz40, grouped= grouped, load= load, outcome= failure)
#b30hz40_stdev.describe()

nrows_raw= 94464    nrows= 944    Number of dropped rows of grouped=  64.0 %

           a1        a2        a3        a4  load  failure
1    3.462722  2.885941  2.067547  2.892010  40.0        1
2    5.467886  5.081921  3.771765  3.134318  40.0        1
3    4.505397  4.474604  4.263436  6.134662  40.0        1
4    4.780326  4.763837  3.188865  3.547939  40.0        1
5    4.761832  4.120222  3.800517  4.511873  40.0        1
..        ...       ...       ...       ...   ...      ...
940  4.977273  5.211589  4.640998  5.107940  40.0        1
941  3.750218  3.727502  2.925126  3.834416  40.0        1
942  5.959528  4.821340  3.998001  5.833581  40.0        1
943  5.568813  4.956242  4.517337  5.890560  40.0        1
944  6.055547  5.407913  5.564590  6.530028  40.0        1

[944 rows x 6 columns]


In [96]:
# [H.4] DATASET HEALTHY, 40% LOAD
load = 40

# CREATE THE COLUMN FOR 'load'
## Expressed as %
h30hz40['load'] = load*np.ones((len(h30hz40.index),1))

# CREATE THE COLUMN FOR 'failure'
failure = 0
failureArray = np.ones((len(h30hz40.index),1))
h30hz40['failure'] = failureArray

h30hz40_stdev= stdev_features (df = h30hz40, grouped= grouped, load= load, outcome= failure)
#b30hz40_stdev.describe()

nrows_raw= 100608    nrows= 1006    Number of dropped rows of grouped=  8.0 %

            a1        a2        a3        a4  load  failure
1     5.358173  3.520671  3.431255  3.875389  40.0        0
2     3.842883  2.178120  2.994552  2.773154  40.0        0
3     6.096313  3.362406  3.098790  3.593120  40.0        0
4     7.518593  4.727157  3.578056  3.488432  40.0        0
5     4.775180  4.136790  2.963929  3.171000  40.0        0
...        ...       ...       ...       ...   ...      ...
1002  6.624234  3.402220  3.549593  4.042739  40.0        0
1003  6.545162  3.997506  4.473011  3.820186  40.0        0
1004  4.410634  3.689969  3.082019  3.154647  40.0        0
1005  4.796943  3.492424  3.627903  3.419745  40.0        0
1006  6.669897  3.709734  4.668362  3.731572  40.0        0

[1006 rows x 6 columns]


## Load= 50%

In [97]:
# [H.5] DATASET BROKEN, 50% LOAD
load = 50

# CREATE THE COLUMN FOR 'load'
## Expressed as %
b30hz50['load'] = load*np.ones((len(b30hz50.index),1))

# CREATE THE COLUMN FOR 'failure'
##   0 is healthy
##   1 is broken
failure = 1
failureArray = np.ones((len(b30hz50.index),1))
b30hz50['failure'] = failureArray
#b30hz50.describe()

b30hz50_stdev= stdev_features (df = b30hz50, grouped= grouped, load= load, outcome= failure)
#b30hz50_stdev.describe()

nrows_raw= 94208    nrows= 942    Number of dropped rows of grouped=  8.0 %

           a1        a2        a3        a4  load  failure
1    4.074277  3.959726  3.714255  4.512339  50.0        1
2    5.187320  5.233059  5.293654  5.405332  50.0        1
3    4.017821  4.337115  3.735858  4.022260  50.0        1
4    3.980077  3.805700  3.251535  3.797708  50.0        1
5    5.351070  4.784816  3.856292  3.052571  50.0        1
..        ...       ...       ...       ...   ...      ...
938  5.834674  5.292088  4.617439  4.544466  50.0        1
939  6.022831  5.870030  5.206705  5.151529  50.0        1
940  4.229254  4.223779  4.320060  4.894417  50.0        1
941  5.238284  4.234645  4.469740  5.188221  50.0        1
942  4.550657  4.600452  3.973703  4.136495  50.0        1

[942 rows x 6 columns]


In [98]:
# [H.5] DATASET HEALTHY, 50% LOAD
load = 50

# CREATE THE COLUMN FOR 'load'
## Expressed as %
h30hz50['load'] = load*np.ones((len(h30hz50.index),1))

# CREATE THE COLUMN FOR 'failure'
failure = 0
failureArray = np.ones((len(h30hz50.index),1))
h30hz50['failure'] = failureArray

h30hz50_stdev= stdev_features (df = h30hz50, grouped= grouped, load= load, outcome= failure)
#b30hz50_stdev.describe()

nrows_raw= 110848    nrows= 1108    Number of dropped rows of grouped=  48.0 %

             a1        a2        a3        a4  load  failure
1      7.666688  4.676215  4.684783  6.325950  50.0        0
2      6.271764  4.564287  4.188379  4.725669  50.0        0
3      5.494505  3.514439  3.378873  3.782597  50.0        0
4      6.088557  3.716829  3.448634  4.096078  50.0        0
5      8.231261  4.582871  4.361992  4.395931  50.0        0
...         ...       ...       ...       ...   ...      ...
1104   5.710045  3.067117  2.864667  3.543264  50.0        0
1105   7.884472  5.187024  4.108225  4.509269  50.0        0
1106   8.277364  5.268922  4.597323  5.016392  50.0        0
1107   6.084638  2.649315  2.890624  4.203750  50.0        0
1108  10.949044  4.594270  5.857479  5.621844  50.0        0

[1108 rows x 6 columns]


## Load= 60%

In [99]:
# [H.6] DATASET BROKEN, 60% LOAD
load = 60

# CREATE THE COLUMN FOR 'load'
## Expressed as %
b30hz60['load'] = load*np.ones((len(b30hz60.index),1))

# CREATE THE COLUMN FOR 'failure'
##   0 is healthy
##   1 is broken
failure = 1
failureArray = np.ones((len(b30hz60.index),1))
b30hz60['failure'] = failureArray
#b30hz60.describe()

b30hz60_stdev= stdev_features (df = b30hz60, grouped= grouped, load= load, outcome= failure)
#b30hz60_stdev.describe()

nrows_raw= 95488    nrows= 954    Number of dropped rows of grouped=  88.0 %

           a1        a2        a3        a4  load  failure
1    4.811947  5.181737  3.748594  3.416334  60.0        1
2    6.711825  4.934653  3.419606  3.703703  60.0        1
3    3.080348  3.764844  2.368740  4.108480  60.0        1
4    4.204862  3.860004  2.947117  3.736848  60.0        1
5    4.514904  4.520084  4.345890  4.359357  60.0        1
..        ...       ...       ...       ...   ...      ...
950  5.430603  4.882351  4.281057  5.610203  60.0        1
951  4.125759  4.726249  3.501445  4.679852  60.0        1
952  4.948803  3.119529  2.444997  2.954189  60.0        1
953  6.600326  4.403814  3.371044  5.253352  60.0        1
954  4.623346  4.628486  3.426586  4.164611  60.0        1

[954 rows x 6 columns]


In [100]:
# [H.6] DATASET HEALTHY, 60% LOAD

# CREATE THE COLUMN FOR 'load'
## Expressed as %
h30hz60['load'] = load*np.ones((len(h30hz60.index),1))

# CREATE THE COLUMN FOR 'failure'
failure = 0
failureArray = np.ones((len(h30hz60.index),1))
h30hz60['failure'] = failureArray

h30hz60_stdev= stdev_features (df = h30hz60, grouped= grouped, load= load, outcome= failure)
#b30hz60_stdev.describe()

nrows_raw= 99840    nrows= 998    Number of dropped rows of grouped=  40.0 %

           a1        a2        a3        a4  load  failure
1    9.662054  5.178468  5.608709  5.423264  60.0        0
2    8.053081  4.092840  4.227479  4.570541  60.0        0
3    8.419113  4.718729  5.113373  6.149591  60.0        0
4    7.912768  4.251369  4.565497  5.937314  60.0        0
5    5.139398  3.317271  2.707007  3.993794  60.0        0
..        ...       ...       ...       ...   ...      ...
994  8.780221  4.098674  5.118693  5.272129  60.0        0
995  5.881403  4.178763  3.600356  4.426720  60.0        0
996  6.174388  3.822195  3.892953  4.499992  60.0        0
997  5.264629  3.853796  3.682252  3.649266  60.0        0
998  7.209492  3.592217  4.021378  4.398803  60.0        0

[998 rows x 6 columns]


## Load= 70%

In [101]:
# [H.7] DATASET BROKEN, 70% LOAD
load = 70

# CREATE THE COLUMN FOR 'load'
## Expressed as %
b30hz70['load'] = load*np.ones((len(b30hz70.index),1))

# CREATE THE COLUMN FOR 'failure'
##   0 is healthy
##   1 is broken
failure = 1
failureArray = np.ones((len(b30hz70.index),1))
b30hz70['failure'] = failureArray
#b30hz70.describe()

b30hz70_stdev= stdev_features (df = b30hz70, grouped= grouped, load= load, outcome= failure)
#b30hz70_stdev.describe()

nrows_raw= 100864    nrows= 1008    Number of dropped rows of grouped=  64.0 %

            a1        a2        a3        a4  load  failure
1     5.504502  4.755241  4.038341  4.872180  70.0        1
2     4.020927  3.418679  2.876730  3.541099  70.0        1
3     4.966444  4.985091  4.919601  4.821201  70.0        1
4     3.839796  4.357150  3.499001  5.472565  70.0        1
5     5.344993  3.773423  4.527043  4.786982  70.0        1
...        ...       ...       ...       ...   ...      ...
1004  4.707994  3.446857  3.049046  4.160162  70.0        1
1005  4.644407  3.570378  3.767121  4.033199  70.0        1
1006  4.771993  3.348961  3.991496  5.122567  70.0        1
1007  4.858836  4.953074  4.528335  5.808374  70.0        1
1008  4.491207  4.414897  3.920422  5.051450  70.0        1

[1008 rows x 6 columns]


In [102]:
# [H.7] DATASET HEALTHY, 70% LOAD
load = 70

# CREATE THE COLUMN FOR 'load'
## Expressed as %
h30hz70['load'] = load*np.ones((len(h30hz70.index),1))

# CREATE THE COLUMN FOR 'failure'
failure = 0
failureArray = np.ones((len(h30hz70.index),1))
h30hz70['failure'] = failureArray

h30hz70_stdev= stdev_features (df = h30hz70, grouped= grouped, load= load, outcome= failure)
#b30hz70_stdev.describe()

nrows_raw= 101376    nrows= 1013    Number of dropped rows of grouped=  76.0 %

             a1        a2        a3        a4  load  failure
1      8.809606  6.797120  4.815198  5.971550  70.0        0
2      8.143851  4.210509  4.137510  5.496099  70.0        0
3      6.601136  3.000749  3.608767  5.517379  70.0        0
4      9.582499  4.519286  5.679526  5.656378  70.0        0
5     10.201630  5.965807  6.589977  6.500685  70.0        0
...         ...       ...       ...       ...   ...      ...
1009   9.849732  3.755824  4.811686  5.507588  70.0        0
1010   9.844470  4.370987  5.254200  5.599210  70.0        0
1011   6.056313  4.141105  3.845646  5.137307  70.0        0
1012   7.510356  4.243818  5.381351  5.542929  70.0        0
1013   4.063881  3.056092  2.598164  2.683162  70.0        0

[1013 rows x 6 columns]


## Load= 80%

In [103]:
# [H.8] DATASET BROKEN, 80% LOAD
load = 80

# CREATE THE COLUMN FOR 'load'
## Expressed as %
b30hz80['load'] = load*np.ones((len(b30hz80.index),1))

# CREATE THE COLUMN FOR 'failure'
##   0 is healthy
##   1 is broken
failure = 1
failureArray = np.ones((len(b30hz80.index),1))
b30hz80['failure'] = failureArray
#b30hz80.describe()

b30hz80_stdev= stdev_features (df = b30hz80, grouped= grouped, load= load, outcome= failure)
#b30hz80_stdev.describe()

nrows_raw= 110335    nrows= 1103    Number of dropped rows of grouped=  35.0 %

            a1        a2        a3        a4  load  failure
1     4.604528  4.032008  3.325304  4.587272  80.0        1
2     4.752443  3.691163  3.288204  4.572145  80.0        1
3     5.353396  3.995737  4.047903  4.478091  80.0        1
4     3.594598  3.542739  3.912459  3.900166  80.0        1
5     4.511902  3.750949  3.634543  3.355099  80.0        1
...        ...       ...       ...       ...   ...      ...
1099  6.387621  5.077033  5.442181  6.305278  80.0        1
1100  4.036438  3.628521  4.247137  4.871524  80.0        1
1101  4.405240  4.449070  4.601878  5.009687  80.0        1
1102  6.117293  5.419937  4.851748  5.062017  80.0        1
1103  3.379544  4.426726  4.008384  3.692929  80.0        1

[1103 rows x 6 columns]


In [104]:
# [H.8] DATASET HEALTHY, 80% LOAD
load = 80

# CREATE THE COLUMN FOR 'load'
## Expressed as %
h30hz80['load'] = load*np.ones((len(h30hz80.index),1))

# CREATE THE COLUMN FOR 'failure'
failure = 0
failureArray = np.ones((len(h30hz80.index),1))
h30hz80['failure'] = failureArray

h30hz80_stdev= stdev_features (df = h30hz80, grouped= grouped, load= load, outcome= failure)
#b30hz80_stdev.describe()

nrows_raw= 99840    nrows= 998    Number of dropped rows of grouped=  40.0 %

            a1        a2        a3        a4  load  failure
1    10.584338  4.187996  5.051522  5.462836  80.0        0
2     9.772850  4.529453  5.290028  4.902815  80.0        0
3     6.545985  4.877882  4.681777  4.848517  80.0        0
4     6.666981  3.585890  4.649248  4.252030  80.0        0
5     5.014805  3.276887  3.527651  3.860534  80.0        0
..         ...       ...       ...       ...   ...      ...
994  11.348938  4.673999  6.019079  6.130194  80.0        0
995   8.013726  4.531021  4.454635  4.362063  80.0        0
996   8.349673  5.129018  5.468724  5.316971  80.0        0
997   8.788454  5.117833  5.363653  5.559049  80.0        0
998   7.272175  4.049968  3.945067  5.454231  80.0        0

[998 rows x 6 columns]


## Load= 90%

In [105]:
# [H.9] DATASET BROKEN, 90% LOAD
load = 90

# CREATE THE COLUMN FOR 'load'
## Expressed as %
b30hz90['load'] = load*np.ones((len(b30hz90.index),1))

# CREATE THE COLUMN FOR 'failure'
##   0 is healthy
##   1 is broken
failure = 1
failureArray = np.ones((len(b30hz90.index),1))
b30hz90['failure'] = failureArray
#b30hz90.describe()

b30hz90_stdev= stdev_features (df = b30hz90, grouped= grouped, load= load, outcome= failure)
#b30hz90_stdev.describe()

nrows_raw= 105728    nrows= 1057    Number of dropped rows of grouped=  28.000000000000004 %

            a1        a2        a3        a4  load  failure
1     4.577819  5.008496  3.802021  4.606855  90.0        1
2     5.779057  5.023367  3.847596  4.602369  90.0        1
3     4.327167  3.699797  2.975056  3.813657  90.0        1
4     4.322838  4.146657  3.503560  4.101523  90.0        1
5     3.880201  4.503873  3.660953  4.476469  90.0        1
...        ...       ...       ...       ...   ...      ...
1053  4.020584  4.213608  3.332272  4.147442  90.0        1
1054  4.842131  6.242672  4.369814  5.759370  90.0        1
1055  4.387001  4.183584  4.093012  5.853607  90.0        1
1056  3.549780  3.318551  3.709936  5.245193  90.0        1
1057  6.136139  5.357831  5.037464  5.913611  90.0        1

[1057 rows x 6 columns]


In [106]:
# [H.9] DATASET HEALTHY, 90% LOAD
load = 90

# CREATE THE COLUMN FOR 'load'
## Expressed as %
h30hz90['load'] = load*np.ones((len(h30hz90.index),1))

# CREATE THE COLUMN FOR 'failure'
failure = 0
failureArray = np.ones((len(h30hz90.index),1))
h30hz90['failure'] = failureArray

h30hz90_stdev= stdev_features (df = h30hz90, grouped= grouped, load= load, outcome= failure)
#b30hz90_stdev.describe()

nrows_raw= 106752    nrows= 1067    Number of dropped rows of grouped=  52.0 %

             a1        a2        a3        a4  load  failure
1      6.892605  4.455469  3.529246  3.843847  90.0        0
2      6.635916  5.525528  4.228780  4.613783  90.0        0
3      8.878290  3.566931  4.624252  5.475219  90.0        0
4      5.777195  3.185480  3.725781  3.688400  90.0        0
5      8.873675  4.151595  5.056517  4.586934  90.0        0
...         ...       ...       ...       ...   ...      ...
1063   9.357705  4.166855  5.627088  5.857498  90.0        0
1064  11.716910  5.356187  6.068804  6.056807  90.0        0
1065  10.526403  5.884372  5.455909  5.788827  90.0        0
1066   9.508300  4.590048  5.310811  5.743670  90.0        0
1067   6.881230  3.288403  4.177912  5.471742  90.0        0

[1067 rows x 6 columns]


# Compare stdev of acceleration traversing % load

In [107]:
print("Load 0%")
print("-------")
stdev_bhz0 = pd.DataFrame(b30hz0_stdev.iloc[:,:-2].mean()).transpose()
stdev_hhz0 = pd.DataFrame(h30hz0_stdev.iloc[:,:-2].mean()).transpose()
print("Broken:\n", stdev_bhz0)
print("Healthy:\n", stdev_hhz0)

Load 0%
-------
Broken:
          a1       a2        a3        a4
0  3.455463  3.79576  3.017348  3.141501
Healthy:
          a1        a2        a3        a4
0  5.644147  3.861692  3.151001  3.231797


In [108]:
print("Load 10%")
print("--------")
stdev_bhz10 = pd.DataFrame(b30hz10_stdev.iloc[:,:-2].mean()).transpose()
stdev_hhz10 = pd.DataFrame(h30hz10_stdev.iloc[:,:-2].mean()).transpose()
print("Broken:\n", stdev_bhz10)
print("Healthy:\n", stdev_hhz10)

Load 10%
--------
Broken:
          a1        a2        a3        a4
0  3.539759  3.855491  3.080835  3.236266
Healthy:
          a1        a2        a3        a4
0  5.709179  3.920917  3.150714  3.329666


In [109]:
print("Load 20%")
print("--------")
stdev_bhz20 = pd.DataFrame(b30hz20_stdev.iloc[:,:-2].mean()).transpose()
stdev_hhz20 = pd.DataFrame(h30hz20_stdev.iloc[:,:-2].mean()).transpose()
print("Broken:\n", stdev_bhz20)
print("Healthy:\n", stdev_hhz20)

Load 20%
--------
Broken:
          a1        a2        a3        a4
0  3.798932  3.895602  3.328675  3.501287
Healthy:
          a1        a2        a3      a4
0  5.986891  3.972294  3.341016  3.5123


In [110]:
print("Load 30%")
print("--------")
stdev_bhz30 = pd.DataFrame(b30hz30_stdev.iloc[:,:-2].mean()).transpose()
stdev_hhz30 = pd.DataFrame(h30hz30_stdev.iloc[:,:-2].mean()).transpose()
print("Broken:\n", stdev_bhz30)
print("Healthy:\n", stdev_hhz30)

Load 30%
--------
Broken:
          a1        a2        a3        a4
0  4.096956  3.950082  3.571447  3.912501
Healthy:
          a1        a2        a3        a4
0  6.637082  4.066547  3.756139  3.956211


In [111]:
print("Load 40%")
print("--------")
stdev_bhz40 = pd.DataFrame(b30hz40_stdev.iloc[:,:-2].mean()).transpose()
stdev_hhz40 = pd.DataFrame(h30hz40_stdev.iloc[:,:-2].mean()).transpose()
print("Broken:\n", stdev_bhz40)
print("Healthy:\n", stdev_hhz40)

Load 40%
--------
Broken:
          a1        a2        a3        a4
0  4.408238  4.129305  3.700573  4.178785
Healthy:
       a1        a2        a3        a4
0  7.196  4.295219  4.065573  4.381915


In [112]:
print("Load 50%")
print("--------")
stdev_bhz50 = pd.DataFrame(b30hz50_stdev.iloc[:,:-2].mean()).transpose()
stdev_hhz50 = pd.DataFrame(h30hz50_stdev.iloc[:,:-2].mean()).transpose()
print("Broken:\n", stdev_bhz50)
print("Healthy:\n", stdev_hhz50)

Load 50%
--------
Broken:
        a1        a2        a3        a4
0  4.6763  4.299412  3.836887  4.452099
Healthy:
         a1        a2       a3        a4
0  7.26809  4.288789  4.17219  4.605306


In [113]:
print("Load 60%")
print("--------")
stdev_bhz60 = pd.DataFrame(b30hz60_stdev.iloc[:,:-2].mean()).transpose()
stdev_hhz60 = pd.DataFrame(h30hz60_stdev.iloc[:,:-2].mean()).transpose()
print("Broken:\n", stdev_bhz60)
print("Healthy:\n", stdev_hhz60)

Load 60%
--------
Broken:
          a1        a2        a3        a4
0  4.985021  4.547715  4.031132  4.784582
Healthy:
          a1        a2        a3        a4
0  7.848711  4.475246  4.403844  5.008874


In [114]:
print("Load 70%")
print("--------")
stdev_bhz70 = pd.DataFrame(b30hz70_stdev.iloc[:,:-2].mean()).transpose()
stdev_hhz70 = pd.DataFrame(h30hz70_stdev.iloc[:,:-2].mean()).transpose()
print("Broken:\n", stdev_bhz70)
print("Healthy:\n", stdev_hhz70)

Load 70%
--------
Broken:
          a1        a2        a3        a4
0  5.174205  4.636795  4.080152  4.799017
Healthy:
          a1        a2       a3        a4
0  7.974429  4.507776  4.52267  5.022316


In [115]:
print("Load 80%")
print("--------")
stdev_bhz80 = pd.DataFrame(b30hz80_stdev.iloc[:,:-2].mean()).transpose()
stdev_hhz80 = pd.DataFrame(h30hz80_stdev.iloc[:,:-2].mean()).transpose()
print("Broken:\n", stdev_bhz80)
print("Healthy:\n", stdev_hhz80)

Load 80%
--------
Broken:
         a1        a2        a3        a4
0  5.21662  4.834102  4.216963  4.951542
Healthy:
          a1        a2        a3        a4
0  8.093019  4.514418  4.587332  5.056444


In [116]:
print("Load 90%")
print("--------")
stdev_bhz90 = pd.DataFrame(b30hz90_stdev.iloc[:,:-2].mean()).transpose()
stdev_hhz90 = pd.DataFrame(h30hz90_stdev.iloc[:,:-2].mean()).transpose()
print("Broken:\n", stdev_bhz90)
print("Healthy:\n", stdev_hhz90)

Load 90%
--------
Broken:
          a1        a2       a3        a4
0  5.157801  4.920361  4.17224  5.100914
Healthy:
          a1        a2       a3        a4
0  8.057251  4.432825  4.57393  5.140485


# Stacked BROKEN dataframe

In [117]:
b30hz_stdev = pd.concat([b30hz0_stdev, b30hz10_stdev, b30hz20_stdev, b30hz30_stdev,b30hz40_stdev,b30hz50_stdev,b30hz60_stdev,b30hz70_stdev,b30hz80_stdev,b30hz90_stdev], axis=0)
b30hz_stdev.describe()

,a1,a2,a3,a4,load,failure
count,10049.000000,10049.000000,10049.000000,10049.000000,10049.000000,10049.0
mean,4.455177,4.294107,3.706363,4.209203,45.478157,1.0
std,1.202618,0.969298,0.947515,1.205743,29.005078,0.0
min,1.468928,1.341292,0.990336,1.111679,0.000000,1.0
25%,3.595718,3.617281,3.020892,3.358830,20.000000,1.0
50%,4.399122,4.256609,3.646947,4.142668,50.000000,1.0
75%,5.221865,4.949478,4.354815,5.006999,70.000000,1.0
max,10.067158,8.071265,7.425122,10.022236,90.000000,1.0


## Export BROKEN dataframe as CSV

In [118]:
b30hz_stdev.to_csv(path + '/b30hz_stdev.csv', index = False)

# Stacked HEALTHY dataframe

In [119]:
h30hz_stdev = pd.concat([h30hz0_stdev, h30hz10_stdev, h30hz20_stdev, h30hz30_stdev,h30hz40_stdev,h30hz50_stdev,h30hz60_stdev,h30hz70_stdev,h30hz80_stdev,h30hz90_stdev], axis=0)
h30hz_stdev.describe()

,a1,a2,a3,a4,load,failure
count,10154.000000,10154.000000,10154.000000,10154.000000,10154.000000,10154.0
mean,7.064944,4.238840,3.987288,4.343511,45.809533,0.0
std,2.231626,1.326401,1.069945,1.247653,28.315587,0.0
min,1.802930,1.272738,1.022471,1.222780,0.000000,0.0
25%,5.496722,3.337468,3.226538,3.453185,20.000000,0.0
50%,6.856787,4.016475,3.916282,4.275365,50.000000,0.0
75%,8.419728,4.846186,4.689546,5.156541,70.000000,0.0
max,23.909199,10.984053,8.273939,10.609798,90.000000,0.0


## Export HEALTHY dataframe as CSV

In [120]:
h30hz_stdev.to_csv(path + '/h30hz_stdev.csv', index = False)